In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3585204271627504497
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5738397696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8359456593362754510
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [2]:
import h5py # .h5 파일을 읽기 위한 패키지
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
device

device(type='cpu')

In [5]:
CFG = {
    'EPOCHS':20,
    'LEARNING_RATE':3e-3,
    'BATCH_SIZE':256,
    'SEED':41
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [7]:
all_df = pd.read_csv('./train.csv')
all_points = h5py.File('./train.h5', 'r')

In [8]:
train_df = all_df.iloc[:int(len(all_df)*0.8)]
val_df = all_df.iloc[int(len(all_df)*0.8):]

In [9]:
class CustomDataset(Dataset):
    def __init__(self, id_list, label_list, point_list):
        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list
        
    def __getitem__(self, index):
        image_id = self.id_list[index]
        
        points = self.point_list[str(image_id)][:]
        image = self.get_vector(points)
        
        if self.label_list is not None:
            label = self.label_list[index]
            return torch.Tensor(image).unsqueeze(0), label
        else:
            return torch.Tensor(image).unsqueeze(0)
    
    def get_vector(self, points, x_y_z=[16, 16, 16]):
        # 3D Points -> [16,16,16]
        xyzmin = np.min(points, axis=0) - 0.001
        xyzmax = np.max(points, axis=0) + 0.001

        diff = max(xyzmax-xyzmin) - (xyzmax-xyzmin)
        xyzmin = xyzmin - diff / 2
        xyzmax = xyzmax + diff / 2

        segments = []
        shape = []

        for i in range(3):
            # note the +1 in num 
            if type(x_y_z[i]) is not int:
                raise TypeError("x_y_z[{}] must be int".format(i))
            s, step = np.linspace(xyzmin[i], xyzmax[i], num=(x_y_z[i] + 1), retstep=True)
            segments.append(s)
            shape.append(step)

        n_voxels = x_y_z[0] * x_y_z[1] * x_y_z[2]
        n_x = x_y_z[0]
        n_y = x_y_z[1]
        n_z = x_y_z[2]

        structure = np.zeros((len(points), 4), dtype=int)
        structure[:,0] = np.searchsorted(segments[0], points[:,0]) - 1
        structure[:,1] = np.searchsorted(segments[1], points[:,1]) - 1
        structure[:,2] = np.searchsorted(segments[2], points[:,2]) - 1

        # i = ((y * n_x) + x) + (z * (n_x * n_y))
        structure[:,3] = ((structure[:,1] * n_x) + structure[:,0]) + (structure[:,2] * (n_x * n_y)) 

        vector = np.zeros(n_voxels)
        count = np.bincount(structure[:,3])
        vector[:len(count)] = count

        vector = vector.reshape(n_z, n_y, n_x)
        return vector

    def __len__(self):
        return len(self.id_list)

In [10]:
train_dataset = CustomDataset(train_df['ID'].values, train_df['label'].values, all_points)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['ID'].values, val_df['label'].values, all_points)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [11]:
class Conv3dModel(nn.Module):
    def __init__(self):
        super(Conv3dModel,self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(1,8,3),
            nn.ReLU(),
            nn.BatchNorm3d(8),
            nn.Dropout(0.3),
            nn.Conv3d(8,32,3),
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.Dropout(0.3),
            nn.MaxPool3d(4),
            nn.Conv3d(32,32,3),
            nn.ReLU(),
        )
        self.classifier = nn.Linear(32,10)

    def forward(self,x):
        x = self.feature_extract(x)
        x = x.view(x.size()[0],-1)
        x = self.classifier(x)
        return x

In [12]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    best_score = 0
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for data, label in tqdm(iter(train_loader)):
            data, label = data.float().to(device), label.long().to(device)
            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        if scheduler is not None:
            scheduler.step()
            
        val_loss, val_acc = validation(model, criterion, val_loader, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss)}] Val Loss : [{val_loss}] Val ACC : [{val_acc}]')
        
        if best_score < val_acc:
            best_score = val_acc
            torch.save(model.state_dict(), './best_model.pth')

In [13]:
def validation(model, criterion, val_loader, device):
    model.eval()
    true_labels = []
    model_preds = []
    val_loss = []
    with torch.no_grad():
        for data, label in tqdm(iter(val_loader)):
            data, label = data.float().to(device), label.long().to(device)
            
            model_pred = model(data)
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
    
    return np.mean(val_loss), accuracy_score(true_labels, model_preds)

In [15]:
model = Conv3dModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [1.55833196146473] Val Loss : [1.1292143046855927] Val ACC : [0.6703]


  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch : [19] Train Loss : [0.14634553596946845] Val Loss : [0.18312484435737134] Val ACC : [0.9437]


  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch : [20] Train Loss : [0.1404470651392724] Val Loss : [0.20835764482617378] Val ACC : [0.9403]


In [16]:
test_df = pd.read_csv('./sample_submission.csv')
test_points = h5py.File('./test.h5', 'r')

In [17]:
test_dataset = CustomDataset(test_df['ID'].values, None, test_points)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [18]:
checkpoint = torch.load('./best_model.pth')
model = Conv3dModel()
model.load_state_dict(checkpoint)
model.eval()

Conv3dModel(
  (feature_extract): Sequential(
    (0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): ReLU()
    (2): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Conv3d(8, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (5): ReLU()
    (6): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): MaxPool3d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (9): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (10): ReLU()
  )
  (classifier): Linear(in_features=32, out_features=10, bias=True)
)

In [19]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for data in tqdm(iter(test_loader)):
            data = data.float().to(device)
            
            batch_pred = model(data)
            
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [20]:
preds = predict(model, test_loader, device)
preds

  0%|          | 0/157 [00:00<?, ?it/s]

[1,
 3,
 2,
 4,
 9,
 9,
 5,
 5,
 4,
 5,
 4,
 3,
 1,
 4,
 7,
 0,
 4,
 9,
 5,
 5,
 5,
 5,
 3,
 6,
 5,
 2,
 5,
 7,
 3,
 7,
 6,
 5,
 7,
 5,
 1,
 0,
 3,
 5,
 5,
 9,
 8,
 3,
 8,
 1,
 4,
 5,
 8,
 3,
 2,
 3,
 5,
 1,
 5,
 5,
 4,
 9,
 3,
 5,
 5,
 6,
 5,
 1,
 2,
 5,
 1,
 5,
 2,
 7,
 1,
 5,
 6,
 3,
 2,
 3,
 5,
 2,
 1,
 8,
 7,
 6,
 1,
 9,
 5,
 1,
 2,
 1,
 5,
 7,
 9,
 2,
 2,
 1,
 8,
 7,
 6,
 3,
 4,
 5,
 5,
 3,
 7,
 7,
 0,
 8,
 2,
 2,
 2,
 6,
 5,
 8,
 3,
 0,
 5,
 8,
 1,
 3,
 0,
 3,
 8,
 1,
 2,
 5,
 7,
 5,
 5,
 9,
 6,
 4,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 1,
 5,
 1,
 5,
 9,
 1,
 5,
 5,
 5,
 7,
 6,
 8,
 6,
 7,
 0,
 5,
 5,
 0,
 5,
 7,
 2,
 1,
 5,
 5,
 4,
 5,
 4,
 4,
 5,
 1,
 3,
 8,
 4,
 4,
 1,
 1,
 2,
 5,
 1,
 9,
 7,
 3,
 0,
 7,
 5,
 8,
 9,
 4,
 8,
 0,
 6,
 0,
 3,
 1,
 8,
 8,
 5,
 6,
 3,
 6,
 5,
 3,
 6,
 8,
 3,
 5,
 1,
 5,
 6,
 5,
 7,
 0,
 1,
 5,
 5,
 8,
 8,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 9,
 2,
 2,
 1,
 3,
 1,
 2,
 1,
 5,
 8,
 6,
 5,
 5,
 4,
 3,
 5,
 5,
 7,
 6,
 9,
 6,
 6,
 8,
 0,
 7,
 4,
 1,
 4,
 5,
 3,
 2,


In [21]:
import plotly.express as px
test_all = h5py.File('./test.h5', 'r')

In [24]:
all_df

,ID,label
0,0,5
1,1,0
2,2,4
3,3,1
4,4,9
...,...,...
49995,49995,5
49996,49996,0
49997,49997,8
49998,49998,4


In [27]:
key = '3'

x = all_points[key][:, 0]
y = all_points[key][:, 1]
z = all_points[key][:, 2]

px.scatter_3d(x=x, y=y, z=z, opacity = 0.8)

In [38]:
key = '53000'
print(preds[3000]) #dataset의 시작이 50000차이가 나기 때문에 53000-50000 = 3000번의 인덱스로 확인

x = test_all[key][:, 0]
y = test_all[key][:, 1]
z = test_all[key][:, 2]

px.scatter_3d(x=x, y=y, z=z, opacity = 0.8)

7
